# Exploratory Analysis of the MLS

In [65]:
import requests
import json
import pandas as pd
from time import sleep
from datetime import datetime

In [2]:
pd.set_option('display.max_columns', 40)

In [467]:
amer_soccer_mls_url = "https://app.americansocceranalysis.com/api/v1/mls/"
fotmob_url = 'https://www.fotmob.com'

In [18]:
url = amer_soccer_mls_url + 'teams'
r = requests.get(url)

In [19]:
teams = pd.read_json(json.dumps(r.json()))
teams.head()

,team_id,team_name,team_short_name,team_abbreviation
0,0KPqjA456v,San Jose Earthquakes,San Jose,SJE
1,19vQ2095K6,New England Revolution,New England,NER
2,4wM42l4qjB,Chivas USA,Chivas,CHV
3,9z5k7Yg5A3,Philadelphia Union,Philadelphia,PHI
4,a2lqR4JMr0,Real Salt Lake,Salt Lake,RSL


In [22]:
# build list of teams to iterate through
team_ids = teams['team_id'].unique()

In [56]:
# extract all matches and combine into a single dataframe
url = amer_soccer_mls_url + 'games'
all_matches = pd.DataFrame()
for team_id in team_ids:
    team_name = teams.team_name.loc[teams.team_id==team_id].item()
    print("Pulling matches for:", team_name)
    params = {'team_id':team_id}
    r = requests.get(url,params)
    tmp_df = pd.read_json(json.dumps(r.json()), convert_dates=False) # has issues with some results
    tmp_df['source_team_id'] = team_id
    all_matches = all_matches.append(tmp_df)
    sleep(0.5)
print("Finished! Found",len(all_matches),"matches.")

Pulling matches for: San Jose Earthquakes
Pulling matches for: New England Revolution
Pulling matches for: Chivas USA
Pulling matches for: Philadelphia Union
Pulling matches for: Real Salt Lake
Pulling matches for: New York Red Bulls
Pulling matches for: CF Montréal
Pulling matches for: D.C. United
Pulling matches for: Los Angeles FC
Pulling matches for: Austin FC
Pulling matches for: Seattle Sounders FC
Pulling matches for: Orlando City SC
Pulling matches for: LA Galaxy
Pulling matches for: Atlanta United FC
Pulling matches for: Toronto FC
Pulling matches for: Minnesota United FC
Pulling matches for: Vancouver Whitecaps FC
Pulling matches for: FC Dallas
Pulling matches for: Columbus Crew
Pulling matches for: Charlotte FC
Pulling matches for: FC Cincinnati
Pulling matches for: Colorado Rapids
Pulling matches for: New York City FC
Pulling matches for: Nashville SC
Pulling matches for: Portland Timbers FC
Pulling matches for: Chicago Fire FC
Pulling matches for: Houston Dynamo FC
Pulling

In [59]:
# save locally
all_matches.to_csv('MLS_All_Matches.csv',header=True)

In [60]:
# take a peek
all_matches.head()

,game_id,date_time_utc,home_score,away_score,home_team_id,away_team_id,referee_id,stadium_id,home_manager_id,away_manager_id,expanded_minutes,season_name,matchday,attendance,knockout_game,last_updated_utc,extra_time,penalties,home_penalties,away_penalties,source_team_id
0,OlMloN20QL,2022-03-13 00:30:00 UTC,2,0,9z5k7Yg5A3,0KPqjA456v,2lqRymD5r0,NWMW8ZN5lz,0Oq6vPrq6D,2vQ10jwqrA,104,2022,3,0,False,2022-03-13 03:03:16 UTC,NaN,NaN,NaN,NaN,0KPqjA456v
1,wvq9RAPwQW,2022-03-05 22:30:00 UTC,3,3,0KPqjA456v,mvzqoLZQap,wvq9vKlQWn,Vj58W84M8n,2vQ10jwqrA,NPqxkDY59d,103,2022,2,12888,False,2022-03-09 14:38:55 UTC,NaN,NaN,NaN,NaN,0KPqjA456v
2,4JMA7YK1qK,2022-02-26 23:00:00 UTC,1,3,0KPqjA456v,a2lqRX2Mr0,Pk5LRpDQOW,Vj58W84M8n,2vQ10jwqrA,9z5kKNb5A3,101,2022,1,16146,False,2022-03-02 08:49:02 UTC,NaN,NaN,NaN,NaN,0KPqjA456v
3,OlMlaRwy5L,2021-11-07 23:00:00 UTC,1,1,0KPqjA456v,mKAqBBmqbg,2lqRymD5r0,Vj58W84M8n,2vQ10jwqrA,2vQ1m6bQrA,99,2021,35,0,False,2021-11-11 07:49:35 UTC,NaN,NaN,NaN,NaN,0KPqjA456v
4,eVq3J8Vn5W,2021-10-30 19:30:00 UTC,3,4,a2lqR4JMr0,0KPqjA456v,4JMAYlPMKg,e7MzlRjqr0,4wM4vOAqjB,2vQ10jwqrA,99,2021,34,15553,False,2021-11-02 08:56:51 UTC,NaN,NaN,NaN,NaN,0KPqjA456v


In [211]:
# convert the dates
all_matches['date_time_utc'] = pd.to_datetime(all_matches['date_time_utc'] , infer_datetime_format=True)  
# recognize it as UTC
all_matches['date_time_utc'] = all_matches['date_time_utc'].dt.tz_localize('UTC')
# convert to PST to match timezone for future processing
all_matches['date_time_est'] = all_matches['date_time_utc'].dt.tz_convert('US/Pacific')
# format the date into string for future processing
all_matches['date_est_formatted'] = all_matches['date_time_est'].dt.strftime("%Y%m%d")

In [212]:
# checking out the data goes
match_dates = all_matches.date_est_formatted.unique()
match_dates.sort()
print("Earliest match:", min(match_dates))
print("Latest match:", max(match_dates))
print("Unique dates:", len(match_dates))

Earliest match: 20130302
Latest match: 20220312
Unique dates: 952


In [425]:
# save referee data
url = amer_soccer_mls_url + 'referees'
r = requests.get(url)
referees = pd.read_json(json.dumps(r.json()))
referees.to_csv('MLS_Referees.csv',header=True)

In [427]:
# save team data
url = amer_soccer_mls_url + 'teams'
r = requests.get(url)
teams = pd.read_json(json.dumps(r.json()))
teams.to_csv('MLS_teams.csv',header=True)

In [429]:
# save manager data
url = amer_soccer_mls_url + 'managers'
r = requests.get(url)
managers = pd.read_json(json.dumps(r.json()))
managers.to_csv('MLS_managers.csv',header=True)

In [432]:
# save stadium data
url = amer_soccer_mls_url + 'stadia'
r = requests.get(url)
stadiums = pd.read_json(json.dumps(r.json()))
stadiums.to_csv('MLS_managers.csv',header=True)

### Fotmob seems to have an API
https://github.com/bgrnwd/fotmob/blob/main/docs/api.md

https://www.fotmob.com/

In [266]:
len(match_dates_beyond_2015)

733

In [275]:
# Fotmob doesnt have data prior to 2015, so we will make a new list for >2015
match_dates_beyond_2015 = match_dates[match_dates > '20150000']
Fotmob_matchesby_date_url = fotmob_url + '/matches?date='
fotmob_matches = pd.DataFrame()
current_query = 0
total_queries = len(match_dates_beyond_2015)
for match_day in match_dates_beyond_2015:
    # get that day's matches
    current_query +=1
    print('Working on:',match_day, end=' | ')
    url = Fotmob_matchesby_date_url + match_day
    r = requests.get(url)
    
    # Extact MLS matches only, the primary ID = 130. Swap this out for other leagues
    # Need to check which result is MLS using primary ID 130
    league_cnt = len(r.json()['leagues'])
    i = 0
    for i in range(league_cnt):
        if r.json()['leagues'][i]['primaryId'] == 130:
            # i will retain the proper league ID
            break
    
    # load MLS matches from that day
    tmp_matches = pd.json_normalize(r.json()['leagues'][0]['matches'])
    print(len(tmp_matches),'matches found | ',"{:.1%}".format(current_query / total_queries),'complete')
    fotmob_matches = fotmob_matches.append(tmp_matches)
    sleep(0.15) # just in case there are rate limits
    
print('Total Matches:', len(fotmob_matches))

Working on: 20150306 | 1 matches found |  0.1% complete
Working on: 20150307 | 6 matches found |  0.3% complete
Working on: 20150308 | 3 matches found |  0.4% complete
Working on: 20150313 | 1 matches found |  0.5% complete
Working on: 20150314 | 5 matches found |  0.7% complete
Working on: 20150315 | 2 matches found |  0.8% complete
Working on: 20150321 | 6 matches found |  1.0% complete
Working on: 20150322 | 1 matches found |  1.1% complete
Working on: 20150328 | 3 matches found |  1.2% complete
Working on: 20150329 | 3 matches found |  1.4% complete
Working on: 20150403 | 1 matches found |  1.5% complete
Working on: 20150404 | 5 matches found |  1.6% complete
Working on: 20150405 | 2 matches found |  1.8% complete
Working on: 20150408 | 1 matches found |  1.9% complete
Working on: 20150410 | 1 matches found |  2.0% complete
Working on: 20150411 | 6 matches found |  2.2% complete
Working on: 20150412 | 2 matches found |  2.3% complete
Working on: 20150416 | 4 matches found |  2.5% c

In [276]:
# save locally
fotmob_matches.to_csv('fotmob_matches.csv',header=True)
fotmob_matches.head()

,id,leagueId,time,statusId,tournamentStage,timeTS,tv,home.id,home.score,home.name,home.longName,away.id,away.score,away.name,away.longName,status.finished,status.started,status.cancelled,status.scoreStr,status.startDateStr,status.startDateStrShort,status.reason.short,status.reason.long,status.whoLostOnAggregated,status.aggregatedStr,status.startTimeStr
0,1883624,838440,07.03.2015 04:20,6,1,1425669600000,None,6637,2,LA Galaxy,LA Galaxy,6397,0,Chicago,Chicago Fire,True,True,False,2 - 0,"Mar 6, 2015",6. Mar.,FT,Full-Time,,NaN,NaN
0,1883625,838440,07.03.2015 21:10,6,1,1425730200000,None,6602,1,DC United,DC United,307692,0,Montreal Impact,Montreal Impact,True,True,False,1 - 0,"Mar 7, 2015",7. Mar.,FT,Full-Time,,NaN,NaN
1,1883631,838440,07.03.2015 22:05,6,1,1425733500000,None,191716,0,Philadelphia,Philadelphia Union,8314,0,Colorado,Colorado Rapids,True,True,False,0 - 0,"Mar 7, 2015",7. Mar.,FT,Full-Time,,NaN,NaN
2,1883626,838440,08.03.2015 00:05,6,1,1425740700000,None,307691,1,Vancouver,Vancouver Whitecaps,56453,3,Toronto,Toronto FC,True,True,False,1 - 3,"Mar 7, 2015",7. Mar.,FT,Full-Time,,NaN,NaN
3,1883633,838440,08.03.2015 02:40,6,1,1425750000000,None,7949,1,FC Dallas,FC Dallas,6603,0,San Jose,San Jose Earthquakes,True,True,False,1 - 0,"Mar 7, 2015",7. Mar.,FT,Full-Time,,NaN,NaN


In [302]:
print(len(fotmob_matches),"matches")

2880 matches


In [298]:
fotmob_matches['home.name'].value_counts()

Toronto             102
Philadelphia        101
Portland             97
Seattle Sounders     97
Houston              96
                   ... 
CS Fola Esch          1
Trepca 89             1
Cracovia              1
FK Spartaks           1
Rostov                1
Name: home.name, Length: 392, dtype: int64

In [303]:
# looks like we have a lot of non MLS teams... may be the 130 code is used in other competitions? 
# limit to just the games where an MLS team is hosting
mls_team_names = ['Toronto', 'Philadelphia', 'Seattle Sounders', 'Houston', 'Portland', 'Orlando', 'Columbus', 'LA Galaxy'
 ,'Vancouver', 'DC United', 'San Jose', 'New England', 'Salt Lake', 'Colorado', 'Sporting KC', 'NYCFC'
 , 'Chicago', 'NY Red Bulls', 'Atlanta United', 'Dallas', 'Minnesota', 'Montreal', 'LAFC', 'FC Dallas'
 , 'Montreal Impact', 'Cincinnati', 'Nashville SC', 'Inter Miami CF', 'Austin FC']

mls_fotmob_matches = fotmob_matches.loc[fotmob_matches['home.name'].isin(mls_team_names)]
print(len(mls_fotmob_matches),"mls matches")

2115 mls matches


In [306]:
# Need a unique list of match IDs, luckily 100% of dupes are cancelled matches
# We'll also exclude the cancelled and never rescheduled matches at the same time
match_ids = mls_fotmob_matches['id'].loc[mls_fotmob_matches['status.cancelled']==False].unique()
print(len(match_ids),"unique matches")

2022 unique matches


In [421]:
def extract_events(r):
    '''r = API response object'''
    events_df = pd.json_normalize(r.json()['content']['matchFacts']['events']['events'])
    # append match_ID and team_id for future joins, don't care about swaps so leaving in list form
    events_df['match_id'] = r.json()['general']['matchId']
    events_df.loc[(events_df['isHome'] == True ),'teamid'] = r.json()['general']['homeTeam']['id']
    events_df.loc[(events_df['isHome'] == False),'teamid'] = r.json()['general']['awayTeam']['id']
    return events_df

In [415]:
def extract_starting_players(r):
    '''r = API response object'''
    starting_players_df = pd.DataFrame()
    # loop through each team (0=home, 1=away)
    i=0
    for i in range(2):
        j=0
        # loop through each rank of players
        for j in range(len(r.json()['content']['lineup']['lineup'][i]['players'])):
            df = pd.json_normalize(r.json()['content']['lineup']['lineup'][i]['players'][j])
            starting_players_df = starting_players_df.append(df)
    # append match_id and team_id for future joining
    starting_players_df['match_id'] = r.json()['general']['matchId']
    starting_players_df.loc[(starting_players_df['isHomeTeam'] == True ),'teamid'] = r.json()['general']['homeTeam']['id']
    starting_players_df.loc[(starting_players_df['isHomeTeam'] == False),'teamid'] = r.json()['general']['awayTeam']['id']
    return starting_players_df

In [448]:
def extract_stats(r):
    '''r = API response object'''
    stats_df = pd.DataFrame()
    # check if stat data exists
    if r.json()['content']['stats'] == None:
        return stats_df
    i=0
    for i in range(len(r.json()['content']['stats']['stats'])):
        df = pd.json_normalize(r.json()['content']['stats']['stats'][i]['stats'])
        # split stats into home and away
        df[['home','away']] = pd.DataFrame(df.stats.tolist(), index= df.index)
        stats_df = stats_df.append(df)
    # append match_id for future joining
    stats_df['match_id'] = r.json()['general']['matchId']
    return stats_df

In [ ]:
%%time
Fotmob_matches_details_url = fotmob_url + '/matchDetails?matchId='
#all_events = pd.DataFrame()
#all_stats = pd.DataFrame()
#all_players = pd.DataFrame()
current_match = 386
total_matches = len(match_ids)
for match_id in match_ids[387:]:
    url = Fotmob_matches_details_url + str(match_id)
    r = requests.get(url)
    # check for a good response, if not try 1 more time after waiting a second
    if not r.ok:
        sleep(1)
        r = requests.get(url)
        # if still no luck, write the ID to an error file and move on
        if not r.ok:
            with open('match_errors', 'a') as errfile:
                errfile.write(str(match_id))
            continue
    all_events  = all_events.append(extract_events(r))
    all_players = all_players.append(extract_starting_players(r)) 
    all_stats   = all_stats.append(extract_stats(r))
    current_match +=1
    print(current_match," / ",total_matches, "completed - ","{:.1%}".format(current_match / total_matches),end ='\r')
    sleep(0.1) # just in case there are rate limits
print('\n')
print('Completed!')

In [480]:
match_ids[387]

2162026

In [470]:
url = fotmob_url + '/matchDetails?matchId=' + str(match_ids[387])
r = requests.get(url)

In [478]:
all_events.to_csv('all_events.csv',header=True)
all_players.to_csv('all_players.csv',header=True)
all_stats.to_csv('all_stats.csv',header=True)

In [460]:
pd.json_normalize(r.json()['content']['matchFacts']['events']['events'])

,reactKey,timeStr,type,time,overloadTime,profileUrl,overloadTimeStr,isHome,injuredPlayerOut,swap,player.id,player.profileUrl,nameStr,card,player.name
0,46SubstitutionNaN114339true,46,Substitution,46,None,/players/NaN/,False,True,False,"[{'name': 'Cristian Maidana', 'id': '114339', ...",NaN,/players/undefined/,NaN,NaN,NaN
1,55SubstitutionNaN431655false,55,Substitution,55,None,/players/NaN/,False,False,False,"[{'name': 'Dillon Powers', 'id': '431655', 'pr...",NaN,/players/undefined/,NaN,NaN,NaN
2,68Card0undefinedfalse,68,Card,68,None,/players/0/bobby-burling,False,False,NaN,NaN,0.0,/players/0/bobby-burling,Bobby Burling,YellowRed,Bobby Burling
3,74SubstitutionNaN342533false,74,Substitution,74,None,/players/NaN/,False,False,False,"[{'name': 'Shane O'Neill', 'id': '342533', 'pr...",NaN,/players/undefined/,NaN,NaN,NaN
4,85SubstitutionNaN254852true,85,Substitution,85,None,/players/NaN/,False,True,False,"[{'name': 'Zach Pfeffer', 'id': '254852', 'pro...",NaN,/players/undefined/,NaN,NaN,NaN
5,85SubstitutionNaN30383true,85,Substitution,85,None,/players/NaN/,False,True,False,"[{'name': 'Conor Casey', 'id': '30383', 'profi...",NaN,/players/undefined/,NaN,NaN,NaN
6,88SubstitutionNaN431650false,88,Substitution,88,None,/players/NaN/,False,False,False,"[{'name': 'Deshorn Brown', 'id': '431650', 'pr...",NaN,/players/undefined/,NaN,NaN,NaN


In [440]:
#url = fotmob_url + '/matchDetails?matchId=3787294'
url = fotmob_url + '/matchDetails?matchId=1883624'
r = requests.get(url)

In [442]:
if "stats" in r.json()['content']:
    print("true")
else:
    print("false")

true


In [446]:
if r.json()['content']['stats'] == None:
    print("True")
else:
    print("False")

True


In [311]:
r.json()['general']['homeTeam']

{'matchId': '3787294',
 'matchName': 'Los Angeles FC-vs-Portland Timbers_Mon, Mar 7, 2022, 03:00 UTC',
 'matchRound': None,
 'teamColors': {'home': '#000000', 'away': '#004010'},
 'leagueId': 10000002,
 'leagueName': 'Major League Soccer',
 'leagueRoundName': 'Major League Soccer null',
 'parentLeagueId': 130,
 'parentLeagueName': 'MLS',
 'parentLeagueSeason': '2022',
 'homeTeam': {'name': 'Los Angeles FC', 'id': 867280},
 'awayTeam': {'name': 'Portland Timbers', 'id': 307690},
 'matchTimeUTC': 'Mon, Mar 7, 2022, 03:00 UTC',
 'started': True,
 'finished': True}

In [319]:
r.json()['header']

{'teams': [{'name': 'Los Angeles FC',
   'id': 867280,
   'score': 1,
   'imageUrl': 'https://images.fotmob.com/image_resources/logo/teamlogo/867280_small.png',
   'pageUrl': '/teams/867280/overview/los-angeles-fc',
   'fifaRank': None},
  {'name': 'Portland Timbers',
   'id': 307690,
   'score': 1,
   'imageUrl': 'https://images.fotmob.com/image_resources/logo/teamlogo/307690_small.png',
   'pageUrl': '/teams/307690/overview/portland-timbers',
   'fifaRank': None}],
 'status': {'finished': True,
  'started': True,
  'cancelled': False,
  'scoreStr': '1 - 1',
  'startDateStr': 'Mar 6, 2022',
  'startDateStrShort': '6. Mar.',
  'reason': {'short': 'FT', 'long': 'Full-Time'},
  'whoLostOnPenalties': None,
  'whoLostOnAggregated': ''},
 'events': {'homeTeamGoals': {'Fall': [{'reactKey': '90Goal1263370undefinedtrue',
     'timeStr': '90 + 3',
     'type': 'Goal',
     'time': 90,
     'overloadTime': 3,
     'eventId': 8825684,
     'player': {'id': 1263370,
      'name': 'Mamadou Fall',
 

In [441]:
with open('json_data_old.json', 'w') as outfile:
    outfile.write(json.dumps(r.json()))
